# Regresión utilizando una red neuronal (ANN)

El dataset (Training_set), contiene un millón de observaciones de 25 variables, asumimos que son (independientes), y una variable de respuesta "y".
Nuestro objetivo será poder predecir la variable y a partir de x1,x2...,x25.

Para ello utilizaremos Keras para generar un modelo simple pero capáz de realizar está tarea.

Las variables 'x' son numéricas x1,x2...,x25 lo mismo que la variable 'y'.

In [95]:
# Importamos las dependencias que vamos a utilizar
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Tomé esta función del repositorio de Jeff Heaton (Washington Univerity) 
# https://github.com/jeffheaton
# toma un dataframe de pandas y convierte sus elememtos en enteros o flotantes de 32 bits
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [34]:
# cargamos los datos de entenamiento
df=pd.read_csv("TrainingSet.csv")
#print(df.head())

# Eliminamos las variables que no tienen nombre (en este caso index (o rownames))
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#print(df.head())

In [41]:
# Aquí utilizamos la función to_xy, donde dividimos el data frame en dos, X (predictores) & y (variable de respuesta)
# luego convertimos la codificación a tipo flotante de 32 bits

X,y = to_xy(df,"y")

# el warning puede ser ignorado por ahora está usando una función que va a desaparecer

C:\Users\QuirozH\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [42]:
# Creamos un conhjunto de entrenamiento y uno de prueba (30%) de los datos seran del set de prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)


In [43]:
# Verificamos las dimensiones del set de datos creado
X_train.shape[0:]

(700000, 25)

In [96]:
#Aqui definimos el modelo de red neuronal (ANN)
model = Sequential()

# Agregamos capas densas (fully connected), especificando el número de nodos, y en la primera capa el numero de inputs
# usamos la funcion de activación relu
model.add(Dense(25, input_dim=X_train.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2

# Dado que el objetivo es la regresión, la última capa poseé solo un nodo que será la respuesta de predicción
model.add(Dense(1)) # Output

# Aquí definimos la función de error que será minimizada y el algoritmo utilizado para ello, 'adam' es una variante de SGD
model.compile(loss='mean_squared_error', optimizer='adam')

# Aquí comienza en entrenamiento:
# Para cada epoca estamos utilizando todos los datos de entrenamiento, esto podría ser modificado para entrenar por batchs (secciones)
model.fit(X_train,y_train,verbose=2,epochs=10)

Epoch 1/10
 - 16s - loss: 2.9470
Epoch 2/10
 - 16s - loss: 0.1248
Epoch 3/10
 - 16s - loss: 0.1044
Epoch 4/10
 - 16s - loss: 0.0879
Epoch 5/10
 - 16s - loss: 0.0781
Epoch 6/10
 - 16s - loss: 0.0710
Epoch 7/10
 - 16s - loss: 0.0628
Epoch 8/10
 - 16s - loss: 0.0594
Epoch 9/10
 - 16s - loss: 0.0559
Epoch 10/10
 - 16s - loss: 0.0533


In [45]:
# Tomámos los inputs del set de prueba y predecimos su valor de Y 
pred = model.predict(X_test)

print("Shape: {}".format(pred.shape))
print(pred)

Shape: (300000, 1)
[[21.838556]
 [14.0237  ]
 [21.29497 ]
 ...
 [ 9.533099]
 [15.696542]
 [16.45551 ]]


In [46]:
# Tomamos los valores predichos y los comparamos con los valores reales, calculamos una metrica de desempeño de la red

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(mean_squared_error(pred,y_test))
print(f"Final score (RMSE): {score}")


Final score (RMSE): 0.180473193526268


In [48]:
# Mostramos los primeros 10 resultados entre el valor de Y actual y el predicho por la red
for i in range(10):
    print(f"{i+1}. Y-Actual: {y_test[i]}, Y-predicted : {pred[i]}")

1. Y-Actual: [21.86156], Y-predicted : [21.838556]
2. Y-Actual: [13.829164], Y-predicted : [14.0237]
3. Y-Actual: [21.597174], Y-predicted : [21.29497]
4. Y-Actual: [13.7927475], Y-predicted : [13.896361]
5. Y-Actual: [11.677943], Y-predicted : [11.7009]
6. Y-Actual: [15.559005], Y-predicted : [15.524943]
7. Y-Actual: [9.624718], Y-predicted : [9.48198]
8. Y-Actual: [16.590979], Y-predicted : [16.602133]
9. Y-Actual: [7.7023554], Y-predicted : [7.846073]
10. Y-Actual: [10.5246935], Y-predicted : [10.487618]


In [94]:
# cargamos los datos no vistos por el modelo
df_test = pd.read_csv("TestSet.csv")

# eliminamos las variables que no tienen nombre
X_final=df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]

print(X_final.head(3))

         x0        x1        x2        x3        x4        x5        x6  \
0  0.644144  0.380748  0.663048  0.163651  0.962608  0.346662  0.991751   
1  0.286435  0.555613  0.376420  0.026624  0.494149  0.834715  0.366987   
2  0.784654  0.264857  0.339367  0.814370  0.807455  0.493562  0.845312   

         x7        x8        x9    ...          x15       x16       x17  \
0  0.235058  0.585694  0.406690    ...     0.089703  0.195771  0.994194   
1  0.515382  0.472379  0.386152    ...     0.244832  0.697950  0.212081   
2  0.365190  0.047831  0.264146    ...     0.684491  0.324034  0.576812   

        x18       x19       x20       x21       x22       x23       x24  
0  0.235180  0.238986  0.629100  0.734953  0.688344  0.031131  0.902514  
1  0.267293  0.373894  0.573089  0.917358  0.818976  0.163181  0.024155  
2  0.632650  0.596462  0.137827  0.618001  0.493464  0.143612  0.030822  

[3 rows x 25 columns]


In [90]:
# ajustamos con el nuevo dataset 
y_final = model.predict(X_final)

# convertimos a dataframe
y_final = pd.DataFrame(data=y_final, columns=['y'])

print(y_final.shape)

(2500, 1)


In [92]:
#creamos la nueva columna en el data frame
final = final.assign(y=pd.Series(y_final['y']).values)
#print(final)

In [93]:
#exportamos el data frame como CSV
final.to_csv("PredictionsANN.csv")